# Imports

In [ ]:
!pip install autokeras

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf
from autokeras import StructuredDataRegressor

train_path = r'../input/tabular-playground-series-aug-2021/train.csv'
test_path = r'../input/tabular-playground-series-aug-2021/test.csv'
submission_path = r'../input/tabular-playground-series-aug-2021/sample_submission.csv'

print('TF',tf.__version__)

RANDOM_SEED = 69420
import os
import gc
def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything()

In [ ]:
# From https://www.kaggle.com/xhlulu/ranzcr-efficientnet-tpu-training
def auto_select_accelerator():
    TPU_DETECTED = False
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
        TPU_DETECTED =True
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    
    return strategy, TPU_DETECTED

strategy, TPU_DETECTED = auto_select_accelerator()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

In [ ]:
KFOLDS = 10
BATCH_SIZES = 16*KFOLDS
EPOCHS = 20*KFOLDS

In [ ]:
train = pd.read_csv(train_path, index_col=0)
train.head()

In [ ]:
x = gc.collect()

In [ ]:
X = train.drop('loss', axis=1).values
y = train.loss.values

X.shape, y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, stratify=y, shuffle=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
sc = MinMaxScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# from joblib import dump
# dump(sc, 'Temp/full_pipe.joblib')

# Optuna Tuning

In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

from sklearn.model_selection import KFold
import tensorflow.keras.backend as K

def make_layer(x, units, dropout_rate):
    t = tfa.layers.WeightNormalization(tf.keras.layers.Dense(units))(x)
    t = tf.keras.layers.Dense(units, tf.keras.activations.swish)(x)
    t = tf.keras.layers.BatchNormalization()(t)
    t = tf.keras.layers.Dropout(dropout_rate)(t)
    return t

def make_model(data, units, dropout_rates):
    
    inputs = tf.keras.layers.Input(shape=(data.shape[1],))
    x = tf.keras.layers.BatchNormalization()(inputs)

    for i in range(len(units)):
        u = units[i]
        d = dropout_rates[i]
        x = make_layer(x, u, d)
       
    y = tf.keras.layers.Dense(1, 'linear', name='dense_output')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=y)
    model.compile(loss='mse', optimizer='adam', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

def fit_predict(n_splits, x_train, y_train, units, dropout_rates, epochs, x_test, y_test, verbose, random_state):

    histories = []
    scores = []
    y_preds = []
    
    cv = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    for train_idx, valid_idx in cv.split(x_train, y_train):

        x_train_train = x_train[train_idx]
        y_train_train = y_train[train_idx]
        x_train_valid = x_train[valid_idx]
        y_train_valid = y_train[valid_idx]
                
        K.clear_session()
        
        with strategy.scope():
            estimator = make_model(x_train, units, dropout_rates)

        es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=2e-5, patience=5,
                                              verbose=verbose, mode='min', restore_best_weights=True)

        rl = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.7, patience=3, min_lr=1e-5,
                                                  mode='min', verbose=verbose)

        history = estimator.fit(
            x_train_train, y_train_train,
            batch_size=BATCH_SIZES,
            epochs=EPOCHS,
            steps_per_epoch=32,
            callbacks=[es, rl],
            validation_data=(x_train_valid, y_train_valid),
            shuffle=True,
            verbose=verbose
        )
        
        if x_test is not None:
            y_part = estimator.predict(x_test)
            y_preds.append(y_part)

        histories.append(history)
        scores.append(history.history['val_root_mean_squared_error'][-1])
    
    if x_test is not None:
        y_pred = np.mean(y_preds, axis=0)
    else:
        y_pred = None

    score = np.mean(scores)
    
    return y_pred, histories, score


In [ ]:
import optuna
def objective(trial):
    
    n_layers = trial.suggest_int('n_layers', 1, 4)
    
    units = []
    dropout_rates = []
    
    for i in range(n_layers):
        u = trial.suggest_categorical('units_{}'.format(i+1), [1024, 512, 256, 128])
        units.append(u)
        r = trial.suggest_loguniform('dropout_rate_{}'.format(i+1), 0.1, 0.5)
        dropout_rates.append(r)
    
    print('Units:', units, "Dropout rates:", dropout_rates, "Layers:", n_layers)
    
    _, _, score = fit_predict(10, X_train, y_train, units, dropout_rates, 50, X_test, y_test, 0, 42)
    return score

In [ ]:
study = optuna.create_study(
    direction='minimize',
    sampler=optuna.samplers.TPESampler(),
    study_name='Optuna_NN'
)

In [ ]:
x = gc.collect()

In [ ]:
%%time
study.optimize(
    objective,
    timeout=3600*7.5,
    gc_after_trial=True
)

In [ ]:
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
optuna.visualization.plot_optimization_history(study)

# Train Model

In [ ]:
params = study.best_trial.params
params

In [ ]:
n_layers = params['n_layers']
    
units = []
dropout_rates = []

for i in range(n_layers):
    u = params['units'+f'_{i+1}']
    units.append(u)
    r = params['dropout_rate'+f'_{i+1}']
    dropout_rates.append(r)

In [ ]:
model = make_model(X_train, units, dropout_rates)
model.summary()

In [ ]:
%%time
y_pred, histories, score = fit_predict(10, X_train, y_train, units, dropout_rates, 50, X_test, y_test, 1, 42)

In [ ]:
print(f"RMSE of Tuned Neural Network: {score}")

# Submit Model

In [ ]:
test = pd.read_csv(test_path, index_col=0)
test.head()

In [ ]:
submission = pd.read_csv(submission_path, index_col=0)
submission.head()

In [ ]:
test = sc.transform(test)

In [ ]:
submission['loss'] = np.abs(model.predict(test))

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv')